# Introducción

## **Pipeline - Crudo a entidades referenciales**

Para convertir los datos en formato crudo a un esquema de base de datos estrella y gestionar identificadores únicos, debemos implementar un modelo que estructure los datos en tablas de hechos y dimensiones. A continuación, te proporciono una propuesta en Python:

1. **Extracción de datos desde el JSON crudo**.
2. **Transformación a un formato que implemente integridad referencial y use identificadores únicos para cada dimensión**.
3. **Almacenamiento en tablas según un esquema de estrella**.


## **Esquema de Base de Datos Estrella**
- **Dimensiones**:
  - `dim_personas`: Información personal única de cada individuo.
  - `dim_empleos`: Detalles del empleo o cargo público.
  - `dim_educacion`: Historial educativo.
  
- **Hechos**:
  - `fact_declaraciones`: Relaciona una declaración patrimonial con las dimensiones, incluyendo métricas financieras.

- `persona_clave`: Una combinación única basada en nombre, apellidos y correo electrónico institucional.
- `Diccionario persona_ids_cache`: Permite garantizar que cada persona tenga un solo ID único, aunque aparezca en diferentes archivos.

Para evitar duplicados y asegurar que una misma persona tenga el mismo persona_id en todas las declaraciones de distintos años, usamos un diccionario que actua como una especie de "cache" para almacenar los IDs únicos asignados a cada persona. El diccionario tiene como clave una combinación de atributos únicos, como el nombre completo y el correo institucional, y como valor, el persona_id correspondiente.

## **Próximos Pasos**
- Verifica si el esquema de las tablas satisface la necesidad de integridad referencial.
- Adapta el código según las herramientas de base de datos que utilices (PostgreSQL, MySQL, etc.).
- Opcional: Implementa validaciones adicionales para asegurar calidad de datos antes de la carga.

# Pipeline principal

In [ ]:
import pandas as pd
import json
import uuid
import os

# Función para generar IDs únicos
def generate_uuid():
    return str(uuid.uuid4())

# Diccionario para almacenar IDs únicos por persona
persona_ids_cache = {}

# Tablas de Dimensiones
dim_personas = []
dim_empleos = []
dim_educacion = []

# Tabla de Hechos
fact_declaraciones = []

# Procesar todos los archivos JSON en una carpeta
carpeta = "sample_declaraciones"
for archivo in os.listdir(carpeta):
    if archivo.endswith(".json"):
        with open(os.path.join(carpeta, archivo), 'r', encoding='utf-8') as file:
            raw_data = json.load(file)

        for record in raw_data:
            try:
                declaracion_id = generate_uuid()  # Identificador único para la declaración

                # Extraer datos generales de la persona, manejando valores faltantes
                persona_data = record.get('declaracion', {}).get('situacionPatrimonial', {}).get('datosGenerales', {})
                empleos_data = record.get('declaracion', {}).get('situacionPatrimonial', {}).get('datosEmpleoCargoComision', {})
                educacion_data = record.get('declaracion', {}).get('situacionPatrimonial', {}).get('datosCurricularesDeclarante', {}).get('escolaridad', [])
                ingresos_data = record.get('declaracion', {}).get('situacionPatrimonial', {}).get('ingresos', {})

                # Crear una clave única para identificar a una persona
                persona_clave = (
                    persona_data.get('nombre', '').strip().upper(),
                    persona_data.get('primerApellido', '').strip().upper(),
                    persona_data.get('segundoApellido', '').strip().upper(),
                    persona_data.get('correoElectronico', {}).get('institucional', '').strip().lower()
                )

                # Revisar si la persona ya tiene un ID asignado
                if persona_clave not in persona_ids_cache:
                    persona_id = generate_uuid()
                    persona_ids_cache[persona_clave] = persona_id

                    # Insertar en dim_personas
                    dim_personas.append({
                        'persona_id': persona_id,
                        'nombre': persona_data.get('nombre', None),
                        'primer_apellido': persona_data.get('primerApellido', None),
                        'segundo_apellido': persona_data.get('segundoApellido', None),
                        'correo_institucional': persona_data.get('correoElectronico', {}).get('institucional', None),
                    })
                else:
                    persona_id = persona_ids_cache[persona_clave]

                # Insertar en dim_empleos
                empleo_id = generate_uuid()
                dim_empleos.append({
                    'empleo_id': empleo_id,
                    'nivel_gobierno': empleos_data.get('nivelOrdenGobierno', None),
                    'ambito_publico': empleos_data.get('ambitoPublico', None),
                    'nombre_ente_publico': empleos_data.get('nombreEntePublico', None),
                    'cargo': empleos_data.get('empleoCargoComision', None),
                    'funcion_principal': empleos_data.get('funcionPrincipal', None),
                    'fecha_toma_posesion': empleos_data.get('fechaTomaPosesion', None),
                })

                # Insertar en dim_educacion
                for edu in educacion_data:
                    educacion_id = generate_uuid()
                    dim_educacion.append({
                        'educacion_id': educacion_id,
                        'persona_id': persona_id,
                        'nivel': edu.get('nivel', {}).get('valor', None),
                        'institucion': edu.get('institucionEducativa', {}).get('nombre', None),
                        'ubicacion': edu.get('institucionEducativa', {}).get('ubicacion', None),
                        'carrera': edu.get('carreraAreaConocimiento', None),
                        'estatus': edu.get('estatus', None),
                        'fecha_obtencion': edu.get('fechaObtencion', None),
                    })

                # Insertar en fact_declaraciones
                fact_declaraciones.append({
                    'declaracion_id': declaracion_id,
                    'persona_id': persona_id,
                    'empleo_id': empleo_id,
                    'total_ingresos_netos': ingresos_data.get('totalIngresosConclusionNetos', {}).get('valor', None),
                    'moneda': ingresos_data.get('totalIngresosConclusionNetos', {}).get('moneda', None),
                })
            except Exception as e:
                print(f"Error procesando el archivo {archivo}: {e}")

# Crear DataFrames para exportar a una base de datos
df_dim_personas = pd.DataFrame(dim_personas)
df_dim_empleos = pd.DataFrame(dim_empleos)
df_dim_educacion = pd.DataFrame(dim_educacion)
df_fact_declaraciones = pd.DataFrame(fact_declaraciones)

# Guardar los DataFrames en CSVs o integrarlos directamente a una base de datos
df_dim_personas.to_csv('dim_personas.csv', index=False)
df_dim_empleos.to_csv('dim_empleos.csv', index=False)
df_dim_educacion.to_csv('dim_educacion.csv', index=False)
df_fact_declaraciones.to_csv('fact_declaraciones.csv', index=False)

print("Transformación completada y datos exportados.")


Transformación completada y datos exportados.


## Sample de las tablas generadas

In [ ]:
df_dim_personas.head()

,persona_id,nombre,primer_apellido,segundo_apellido,correo_institucional
0,abb419b6-fd56-4dc9-877d-fd9538249f8b,FRED,CERON,SANTILLÁN,xxxxx.xxxxx@xxxxxx
1,64a93a1b-cf04-480e-8e8b-7a25dfdf2f55,ARCELIA,CERVANTES,TELLEZ,arcelia.cervantes@ine.mx
2,c3ecf0b6-6e66-4f94-b61d-3de38de46a70,LAURA,CHIPRÉS,BARRANCO,laura.chipres@ine.mx
3,2da769f1-6595-4b82-9160-e449278b4367,SARA ITZEL,CISNEROS,MARTINEZ,xxxxx.xxxxx@xxxxxx
4,22df4b4f-1ef7-4336-a5aa-b5b4f9a032c9,ADRIANA,CRISPIN,VIEYRA,xxxxx.xxxxx@xxxxxx


In [ ]:
df_dim_empleos.head()

,empleo_id,nivel_gobierno,ambito_publico,nombre_ente_publico,cargo,funcion_principal,fecha_toma_posesion
0,56a624ba-d7b1-4bbb-a9e0-cf65d8407141,FEDERAL,ORGANO_AUTONOMO,Instituto Nacional Electoral,SUPERVISOR ELECTORAL,"COORDINAR, APOYAR Y VERIFICAR TRABAJO DE CAPAC...",2021-01-25
1,050d0f85-3012-4df5-9fd2-c92d4230794a,FEDERAL,ORGANO_AUTONOMO,Instituto Nacional Electoral,CONSEJERO DISTRITAL ELECTORAL,CONSEJERA DISTRITAL ELECTORAL,2020-12-01
2,542bc707-3895-416f-8faf-48db0c3788fc,FEDERAL,ORGANO_AUTONOMO,Instituto Nacional Electoral,Subcoordinador de audio y video D,Dictaminar y verificar el control de calidad d...,2020-12-31
3,8b20b9e8-58dd-4b3f-af4c-a0cd52080588,FEDERAL,ORGANO_AUTONOMO,Instituto Nacional Electoral,Apoyo Administrativo en Junta Distrital,Apoyar en los requerimientos en materia de rec...,2020-12-01
4,90c582c7-6212-4fa2-9266-865d3d2bd2bc,FEDERAL,ORGANO_AUTONOMO,Instituto Nacional Electoral,CONSEJERO DISTRITAL ELECTORAL,CONSEJERA ELECTORAL,2020-12-01


In [ ]:
df_dim_educacion.head()

,educacion_id,persona_id,nivel,institucion,ubicacion,carrera,estatus,fecha_obtencion
0,d68828a4-7737-47f2-8870-d2142a504261,abb419b6-fd56-4dc9-877d-fd9538249f8b,BACHILLERATO,CBTIS,MX,TECNICO EN ADMINISTACION,FINALIZADO,1986-07-05
1,43a9e689-d9db-44f6-8ba7-f95f677ccd72,abb419b6-fd56-4dc9-877d-fd9538249f8b,SECUNDARIA,JOSE MARIA LEZAMA,MX,SECUNDARIA,FINALIZADO,1968-07-06
2,b793eacf-85f4-40e1-950b-22d8b0d4fcef,abb419b6-fd56-4dc9-877d-fd9538249f8b,PRIMARIA,JOSEFINA MARIA VALENCIA,MX,PRIMARIA,FINALIZADO,1965-07-15
3,eccc12a9-9d92-4bb3-a5a4-8ad6b752b390,64a93a1b-cf04-480e-8e8b-7a25dfdf2f55,LICENCIATURA,UNIVERSIDAD PEDAGÓGICA NACIONAL,MX,LICENCIATURA EN EDUCACIÓN,FINALIZADO,2002-07-15
4,4d195b5a-2b58-45eb-b920-02ca0d4aec7b,c3ecf0b6-6e66-4f94-b61d-3de38de46a70,LICENCIATURA,UNIVERSIDAD NACIONAL AUTÓNOMA DE MÉXICO,MX,COMUNICACIÓN Y PERIODISMO,FINALIZADO,2017-05-15


In [ ]:
df_fact_declaraciones.sample(20)

,declaracion_id,persona_id,empleo_id,total_ingresos_netos,moneda
195,d156e088-843c-48cf-aba8-1a001f695d93,ebe81bea-78d9-4d72-ac34-d965b118b35e,07c73141-39d8-4888-97f1-474ca0ecc451,NaN,None
21,c4e9d9b8-bf36-4655-8a62-9f6b576608a3,46cb1e31-92cc-4e87-8f0a-49ff4b77066f,331f01ff-9cd9-4094-8f44-601dd4be2d60,NaN,None
382,7d97e57a-a9de-4f32-ac62-7f1167a4caa3,a52ac1ee-2dce-4e88-b07e-c3ceeabfe58d,aec40170-aa87-4228-8efa-ac46bd8885bf,NaN,None
31,21c16acb-127e-4e97-af80-f670527ec740,a52edb57-7844-4465-a2e9-32d5efc38345,8a0996df-a945-4f44-a114-aa9c6a42aa62,NaN,None
28,19898a10-be0f-410f-a219-fd7561014d78,f7cb0d2f-b21a-4391-8d23-07a9691c705f,bb5b9ab6-dd68-4f64-b3d8-a912d80f999b,-1.0,MXN
359,67632d39-8e5f-4711-b3e6-23cb05d9ac98,82bd0b1d-d794-48a4-b6bf-f7c681566a8e,7854a573-b19d-496a-860f-cea9bc5211f4,NaN,None
219,9a1ae659-d7d7-44ef-8ed1-58da4302516a,08d38a90-e786-46e6-b5f2-bd355ef5e61f,6b69d09a-d7bc-4d30-9c2e-9b3b5d5145fe,-1.0,MXN
259,b91781f7-9ce1-4d7e-9f9d-e182241f5245,491c48cb-3048-4bb2-a20b-a8a23ca3eb41,d80c7b30-1c38-4a05-843a-5830804d2a39,NaN,None
398,79cda734-42eb-4c8b-9d8e-9601682bee3a,9c1972af-b004-464a-9b69-ed0a7f462cdf,ae7594d1-0770-428e-a6aa-de49767d6aa5,NaN,None
317,6b5d3520-5a43-4d04-b240-872be98a6e88,64a93a1b-cf04-480e-8e8b-7a25dfdf2f55,9555a3b6-e777-42b3-b4f7-71e4f1128ed5,NaN,None


# Identificador de posibles nombres duplicados

 Un script que identifica posibles filas duplicadas en df_dim_personas. El enfoque combina varios campos (como nombre, apellidos y correo) para determinar similitudes, y también maneja casos en los que puede faltar información o variar en formato (por ejemplo, nombres incompletos). Utilizaremos la biblioteca fuzzywuzzy para comparar similitudes en los nombres y detectar duplicados con ligeras variaciones.

In [ ]:
!pip install fuzzywuzzy --quiet
!pip install tqdm --quiet

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm

# Cargar el DataFrame (por ejemplo, desde un archivo CSV)
df_dim_personas = pd.read_csv('dim_personas.csv')

# Crear una columna combinada para simplificar la comparación
df_dim_personas['nombre_completo'] = (
    df_dim_personas['nombre'].fillna('') + " " +
    df_dim_personas['primer_apellido'].fillna('') + " " +
    df_dim_personas['segundo_apellido'].fillna('')
).str.strip().str.upper()

# Normalizar correos y eliminar irrelevantes
irrelevant_email = "xxxxx.xxxxx@xxxxxx"
df_dim_personas['correo_institucional'] = df_dim_personas['correo_institucional'].apply(
    lambda x: None if pd.isna(x) or x.strip().lower() == irrelevant_email else x.strip().lower()
)

# Función para detectar duplicados basados en similitudes
def find_possible_duplicates(df, threshold=85):
    """
    Identifica posibles filas duplicadas en un DataFrame basándose en similitudes de texto.

    :param df: DataFrame con la columna 'nombre_completo' para comparar.
    :param threshold: Umbral de similitud (0-100) para considerar nombres como duplicados.
    :return: DataFrame con posibles duplicados.
    """
    duplicates = []
    nombres = df['nombre_completo'].tolist()
    correos = df['correo_institucional'].tolist()

    total = len(nombres)
    with tqdm(total=total, desc="Procesando registros", unit="registro") as pbar:
        for i, nombre in enumerate(nombres):
            for j, comparador in enumerate(nombres):
                if i != j:
                    # Comparar nombres con similitud fuzzy
                    nombre_similarity = fuzz.token_sort_ratio(nombre, comparador)

                    # Comparar correos institucionales si son válidos
                    correo_similarity = (
                        fuzz.partial_ratio(correos[i], correos[j])
                        if correos[i] and correos[j] else 0
                    )

                    if nombre_similarity >= threshold or correo_similarity >= threshold:
                        duplicates.append({
                            'index_1': i,
                            'index_2': j,
                            'nombre_1': nombre,
                            'nombre_2': comparador,
                            'correo_1': correos[i],
                            'correo_2': correos[j],
                            'nombre_similarity': nombre_similarity,
                            'correo_similarity': correo_similarity
                        })
            pbar.update(1)
    return pd.DataFrame(duplicates)

# Buscar posibles duplicados
possible_duplicates = find_possible_duplicates(df_dim_personas)

# Exportar los posibles duplicados a un archivo CSV para revisión manual
possible_duplicates.to_csv('posibles_duplicados.csv', index=False)

print(f"\nSe encontraron {len(possible_duplicates)} posibles duplicados. \nExportados a 'posibles_duplicados.csv'.")

Procesando registros: 100%|██████████| 311/311 [00:16<00:00, 19.05registro/s]


Se encontraron 14 posibles duplicados. 
Exportados a 'posibles_duplicados.csv'.


In [ ]:
possible_duplicates.sort_values(by="nombre_similarity", ascending=False).head()

,index_1,index_2,nombre_1,nombre_2,correo_1,correo_2,nombre_similarity,correo_similarity
1,94,99,ESMERALDA GONZALEZ ORTIZ,CLAUDIA MARIBEL GONZALEZ RAMIREZ,esmeralda.gonzalez@ine.mx,claudia.gonzalezr@ine.mx,57,86
2,99,94,CLAUDIA MARIBEL GONZALEZ RAMIREZ,ESMERALDA GONZALEZ ORTIZ,claudia.gonzalezr@ine.mx,esmeralda.gonzalez@ine.mx,57,86
6,127,133,ORLANDO HERNANDEZ CARRERA,JUAN CARLOS GIOVANY HERNANDEZ GOMEZ,orlando.hernandez@ine.mx,juan.hernandezz@ine.mx,57,88
7,133,127,JUAN CARLOS GIOVANY HERNANDEZ GOMEZ,ORLANDO HERNANDEZ CARRERA,juan.hernandezz@ine.mx,orlando.hernandez@ine.mx,57,88
8,133,143,JUAN CARLOS GIOVANY HERNANDEZ GOMEZ,JOSE DEL CARMEN HERNANDEZ RAMIREZ,juan.hernandezz@ine.mx,josedelcarmen.hernandez@ine.mx,53,86


# **Conclusión**

### Conclusión: Beneficios del Proceso de Transformación

Pasar del formato original o crudo al sistema de datos referenciales, como se logra mediante el código generado, ofrece numerosos beneficios tanto para la calidad de los datos como para su gestión y análisis. Estos beneficios se pueden resumir en los siguientes puntos clave:

---

#### **1. Unicidad y Consistencia de la Información**
- **Evita duplicados:** El uso de identificadores únicos (`UUIDs`) asegura que cada persona, empleo, o dato educativo se registre una sola vez, incluso si aparece en múltiples años o declaraciones.
- **Resolución de inconsistencias:** La consolidación de datos similares (por ejemplo, nombres o apellidos incompletos) mejora la coherencia y elimina ambigüedades.

#### **2. Eficiencia en el Análisis**
- **Facilita la consulta y análisis:** Al estructurar los datos en un esquema de estrella, las relaciones entre hechos y dimensiones son más claras y rápidas de consultar.
- **Reducción de redundancia:** Evitar la duplicación de datos en múltiples registros reduce el tamaño de las tablas y optimiza la gestión de recursos.

#### **3. Escalabilidad y Mantenimiento**
- **Estructura modular:** Separar datos en tablas referenciales (dimensiones) permite agregar o actualizar información sin afectar otros aspectos del sistema.
- **Manejo de cambios:** Es más fácil ajustar o expandir las reglas de negocio, como agregar validaciones o nuevos catálogos de datos.

#### **4. Mejora en la Calidad de Datos**
- **Validación cruzada:** La transformación incluye reglas para asegurar la integridad referencial (por ejemplo, nombres que coincidan con roles o empleos).
- **Identificación de anomalías:** El sistema permite detectar errores, como inconsistencias en declaraciones financieras o patrones sospechosos.

#### **5. Compatibilidad con Sistemas Modernos**
- **Preparación para análisis avanzado:** Este modelo es adecuado para su uso con herramientas de análisis de datos (como BI o ML), ya que estructura la información de manera lógica y eficiente.
- **Interoperabilidad:** Los datos referenciales pueden integrarse fácilmente en sistemas relacionales, como PostgreSQL o MySQL, o en herramientas de big data.

#### **6. Transparencia y Trazabilidad**
- **Seguimiento de datos históricos:** Cada declaración se relaciona con un individuo mediante un identificador único, permitiendo reconstruir y auditar el historial completo.
- **Mayor claridad en datos públicos:** Este esquema facilita responder preguntas específicas (por ejemplo, evolución de ingresos) con datos confiables y estructurados.

---

### En Resumen
La transformación de datos crudos a un sistema de datos referenciales mejora la integridad, la eficiencia, y el potencial de análisis de los datos. Este proceso no solo simplifica la gestión y uso de la información, sino que también establece una base sólida para proyectos más complejos en el futuro, como el análisis de riesgos de corrupción o la evaluación de políticas públicas.